# Classification after Clustering with Agglomerative Clustering

### Import CSV and Required Packages

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import ConfusionMatrixDisplay

warnings.filterwarnings("ignore")
pd.pandas.set_option("display.max_columns", None)
# Create Dataframe
df = pd.read_csv(r"./data_1/clustered_data.csv")
# Print shape of dataset
print(df.shape)

(2240, 22)


**Split X and y**
- Why do we split our data?
> Training Dataset is the part of Original Dataset that we use to train our ML model. The model learns on this data by running the algorithm and maps a function F(x) where “x” in the independent variable (inputs) for “y” where “y” is the dependent variable(output).

In [35]:
X = df.drop("cluster", axis=1) #dropping the target column which is 'cluster'
y = df["cluster"]

## Grid Search

- Why do we use Grid Search?

`GridSearchCV` is a technique to search through the best parameter values from the given set of the grid of parameters. It is basically a cross-validation method. the model and the parameters are required to be fed in. Best parameter values are extracted and then the predictions are made.

## Select the best model
- so here we have some list of the best classification algorithms we imported. Now we will compare each model's score and see which model is performing better than rest of the others

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report,ConfusionMatrixDisplay, \
                            precision_score, recall_score, f1_score, roc_auc_score,roc_curve,confusion_matrix
from sklearn import metrics 

models = {
    "Random Forest": RandomForestClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "Logistic Regression": LogisticRegression(),
     "K-Neighbors Classifier": KNeighborsClassifier(),
        "AdaBoost Classifier": AdaBoostClassifier()
}

- ### We will create a generic function to check each model's performance so that we can compare those

In [40]:
# Create a function which can evaluate models and return a report 
def evaluate_models(X, y, models):
    '''
    This function takes in X and y and models dictionary as input
    It splits the data into Train Test split
    Iterates through the given model dictionary and evaluates the metrics
    Returns: Dataframe which contains report of all models metrics with cost
    '''
    # separate dataset into train and test
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
    

    models_list = []
    scores = []
    
    for i in range(len(list(models))):
        model = list(models.values())[i]
        model.fit(X_train, y_train) # Train model

        # Make predictions
        y_pred = model.predict(X_test)

        score = accuracy_score(y_test,y_pred)
        
        model_name = list(models.keys())[i]
        print(f'---- score for --- {model_name} ----')
        print(f"{score}")
        models_list.append(model_name)
        scores.append(score)
    
    print()
    
    report = pd.DataFrame()
    report['Model_name'] = models_list
    report['Score'] = scores        
    return report

### Let's check the report

In [42]:
report = evaluate_models(X, y, models)

---- score for --- Random Forest ----
0.9553571428571429
---- score for --- Decision Tree ----
0.9486607142857143
---- score for --- Gradient Boosting ----
0.9642857142857143
---- score for --- Logistic Regression ----
0.875
---- score for --- K-Neighbors Classifier ----
0.8236607142857143
---- score for --- AdaBoost Classifier ----
0.9419642857142857



In [43]:
report.sort_values('Score')

,Model_name,Score
4,K-Neighbors Classifier,0.823661
3,Logistic Regression,0.875000
5,AdaBoost Classifier,0.941964
1,Decision Tree,0.948661
0,Random Forest,0.955357
2,Gradient Boosting,0.964286


- ### From the report above we can see that the logistic regression model performed the best, so we will continue training our model using logistic regression algorithm.

### Split into Train and test data

- **Do you know why we split the train and test dataset?**
> The train test split technique can be used for classification and regression problems to test machine learning algorithms. The procedure takes the given dataset and splits it into two subsets: ```Training data/train set:``` it is used to train the algorithm and fit the machine learning model
then we have ```test data/test set``` which is basically a different data for which we know the values but this data was never shown to the model before. Thus if the model after training is performing good on test set as well then we can say that the Machine Learning model is good.

In [46]:
# Separate data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)

X_train

,Age,Education,Marital Status,Parental Status,Children,Income,Total_Spending,Days_as_Customer,Recency,Wines,Fruits,Meat,Fish,Sweets,Gold,Web,Catalog,Store,Discount Purchases,Total Promo,NumWebVisitsMonth
994,71,2,0,1,2,64587.0,108.0,4227,49,66,0,16,0.0,6,20.0,1,1,4,2,0,3
2162,68,2,1,1,1,47320.0,414.0,4757,10,200,19,111,50.0,15,19.0,5,1,8,6,0,6
906,64,2,0,0,0,86429.0,1449.0,4440,10,464,28,556,29.0,18,37.0,7,4,7,0,1,2
572,47,1,0,1,1,38593.0,177.0,4509,42,51,12,49,17.0,24,24.0,4,1,3,3,0,8
1877,67,2,1,0,0,72905.0,1515.0,4379,52,407,81,445,120.5,81,126.5,3,7,9,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,56,2,1,1,2,44078.0,41.0,4230,17,24,1,10,2.0,0,4.0,2,0,3,2,0,5
1095,46,2,0,1,1,61825.0,424.0,4576,56,162,50,100,55.0,30,27.0,4,2,8,1,0,4
1130,74,3,1,1,1,67381.0,957.0,4750,67,815,8,53,11.0,0,70.0,2,2,9,4,1,7
1294,62,4,0,1,2,48918.0,62.0,4062,21,52,0,9,0.0,0,1.0,1,0,4,2,0,4


### Let's do hyperparameter tuning
- **And what's it actually?**

> A Machine Learning model is defined as a mathematical model with a number of parameters that need to be learned from the data. By training a model with existing data, we are able to fit the model parameters. 
However, there is another kind of parameter, known as Hyperparameters, that cannot be directly learned from the regular training process. They are usually fixed before the actual training process begins. These parameters express important properties of the model such as its complexity or how fast it should learn. 

In [ ]:
# # Grid search cross-validation
# from sklearn.model_selection import GridSearchCV

# params = {
#     "n_estimators": np.arange(50, 301, 50),     # number of trees
#     "learning_rate": [0.01, 0.05, 0.1, 0.2],
#     "max_depth": [2, 3, 4, 5],
#     "min_samples_split": [2, 5, 10],
#     "min_samples_leaf": [1, 2, 4],
#     "subsample": [0.6, 0.8, 1.0],
#     "max_features": ["sqrt", "log2", None]
# }

# gdbc=GradientBoostingClassifier()

# gdbc_cv=GridSearchCV(gdbc,params,cv=10)
# gdbc_cv.fit(X_train,y_train)

# print("tuned hpyerparameters :(best parameters) ",gdbc_cv.best_params_)
# print("accuracy :",logreg_cv.best_score_)

### Let's now train the model 

In [48]:
gdbc=GradientBoostingClassifier()

gdbc.fit(X_train,y_train)
y_pred = gdbc.predict(X_test)

In [50]:
X_train.dtypes

Age                     int64
Education               int64
Marital Status          int64
Parental Status         int64
Children                int64
Income                float64
Total_Spending        float64
Days_as_Customer        int64
Recency                 int64
Wines                   int64
Fruits                  int64
Meat                    int64
Fish                  float64
Sweets                  int64
Gold                  float64
Web                     int64
Catalog                 int64
Store                   int64
Discount Purchases      int64
Total Promo             int64
NumWebVisitsMonth       int64
dtype: object

**Initialize model with best parameters**

### Let's check the report now

In [52]:
# best_model = best_lr_model.fit(X_train,y_train)
# y_pred = best_model.predict(X_test)
score = accuracy_score(y_test,y_pred)
cr = classification_report(y_test,y_pred)

print(" Gradient Boosting ")
print("Accuracy Score value: {:.4f}".format(score))
print(cr)

 Gradient Boosting 
Accuracy Score value: 0.9613
              precision    recall  f1-score   support

           0       0.94      0.95      0.94       229
           1       0.98      0.98      0.98       265
           2       0.96      0.96      0.96       178

    accuracy                           0.96       672
   macro avg       0.96      0.96      0.96       672
weighted avg       0.96      0.96      0.96       672



## Confusion matrix of the model
- **What is confusion matrix ?**
> The confusion matrix is a matrix used to determine the performance of the classification models for a given set of test data. It can only be determined if the true values for test data are known. The matrix itself can be easily understood, but the related terminologies may be confusing.

- **Reports**

**We can see, that the model performed pretty well.**
- we have used Gradient Boosting as it performed well than other models
- We got a good accuracy while predicting the test dataset.

### Now save my model till here to save it in the pickle file

In [54]:
from sklearn.pipeline import Pipeline
import pickle
import os

# Ensure artifacts folder exists
os.makedirs("artifacts", exist_ok=True)
# with open("artifacts/preprocessor2.pkl", "rb") as f:
#     preprocessor2 = pickle.load(f)
# Full pipeline = preprocessing + model
full_pipeline = Pipeline([
    # ("preprocessor2", preprocessor2),
    ("model", gdbc)
])

# Save the pipeline
with open("artifacts/customer_segmentation_pipeline.pkl", "wb") as f:
    pickle.dump(full_pipeline, f)

In [56]:
import pickle

In [58]:
with open("artifacts/customer_segmentation_pipeline.pkl","rb") as f:
    model = pickle.load(f)

In [151]:
# score = accuracy_score(y_test,y_pred)
# cr = classification_report(y_test,y_pred)

# print(" Gradient Boosting ")
# print("Accuracy Score value: {:.4f}".format(score))
# print(cr)

 Gradient Boosting 
Accuracy Score value: 0.9598
              precision    recall  f1-score   support

           0       0.94      0.95      0.94       229
           1       0.98      0.98      0.98       265
           2       0.95      0.95      0.95       178

    accuracy                           0.96       672
   macro avg       0.96      0.96      0.96       672
weighted avg       0.96      0.96      0.96       672



In [94]:
import pandas as pd
input_data = {
    "Age": 56,
    "Education": 2,
    "Marital Status": 1,
    "Parental Status": 0,
    "Children": 0,
    "Income": 74000.0,
    "Total_Spending": 1320.0,
    "Days_as_Customer": 4550,
    "Recency": 49,
    "Wines": 590,
    "Fruits": 48,
    "Meat": 370,
    "Fish": 75.0,
    "Sweets": 48,
    "Gold": 65.0,
    "Web": 5,
    "Catalog": 6,
    "Store": 8,
    "Discount Purchases": 1,
    "Total Promo": 1,
    "NumWebVisitsMonth": 3
}


df = pd.DataFrame([input_data])
df

,Age,Education,Marital Status,Parental Status,Children,Income,Total_Spending,Days_as_Customer,Recency,Wines,Fruits,Meat,Fish,Sweets,Gold,Web,Catalog,Store,Discount Purchases,Total Promo,NumWebVisitsMonth
0,56,2,1,0,0,74000.0,1320.0,4550,49,590,48,370,75.0,48,65.0,5,6,8,1,1,3


In [96]:
prediction = model.predict(df)

In [98]:
prediction[0]

2